In [3]:
# Used to track the request processing time
import time

from textwrap import dedent
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

In [4]:
'''
Initialize Llama3 LLM for agents.
This will access local models in the standard installation directory
'''
llm = Ollama(model="NEW_MODEL")

In [6]:
'''
Uncomment the below line to test if the loaded model
works and is indeed a custom model.
'''
llm.invoke("Tell me a joke")

'Hwæt! Hear me, I shall tell thee a jest most merry!\n\nWhy did the wight go to the alehouse?\n\nForsooth, \'twas because he heard they had a "ghoul" time! (get it? ghoul... good... haha)\n\nHwæt! Didst thou find that joke amusing, good sir or madam?'

# Create Agents

In [ ]:
class FunctionTestAgents:

    def variable_name_agent(self):
        return Agent(
            role='Variable Name Agent',
            goal='Ensure that variable names are descriptive, concise, and accurate.',
            backstory=dedent(
                '''
                You're an expect agent that helps developers write clean, efficient code.
                Your job is to ensure that variable names are descriptive, concise, and accurate.
                Do not add comments to the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def code_comment_agent(self):
        return Agent(
            role='Code Comment Agent',
            goal='Add comments to code to explain its purpose and functionality. Only add comments that are necessary for a user to understand the code.',
            backstory=dedent(
                '''
                You're an expect code commentor. Your job is to add comments to the code to explain its purpose and functionality. Only add comments that
                are necessary for a user to understand the code. You're aware that code should be self-explanatory whenever possible, and that comments
                should only be used to explain complex or non-obvious parts of the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def documentation_agent(self):
        return Agent(
            role='Documentation Agent',
            goal='Document code in a way that is easy to understand and follow.',
            backstory=dedent(
                '''
                You're an expert in writing docstrings. Your documentation is clear, concise, accurate, and easy to understand.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

# Create Tasks

In [ ]:
class FunctionTestTasks:

    def improve_variable_names(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Ensure that all variable names are descriptive, concise, and accurate.
                Do not add comments to the code.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with improved variable names
                '''),
                agent=agent
        )

    def add_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Add comments to the code to explain its purpose and functionality.
                Only add comments that are necessary for a user to understand the code. If a line of
                code is self-explanatory, do not add a comment.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added comments
                '''),
                agent=agent
        )

    def review_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code with comments. Review the code and ensure that all comments are
                accurate and helpful. If a comment is not necessary, remove it.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with reviewed comments
                '''),
                agent=agent
        )

    def add_documentation(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Add docstrings to the code to explain its purpose and functionality.
                Ensure the documentation is clear, concise, accurate, and easy to understand.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added documentation
                '''),
                agent=agent
        )

# Main Code

In [ ]:
# Initialize Agents, Tasks
agents = FunctionTestAgents()
tasks = FunctionTestTasks()

# CREATE AGENTS
variable_name_agent = agents.variable_name_agent()
# code_comment_agent = agents.code_comment_agent()
# documentation_agent = agents.documentation_agent()

print("##################################")
print("## Welcome to the Function Crew ##")
print("##################################")

game_request = input("Enter the code you want to improve: ")
start_time = time.perf_counter()

# CREATE TASKS
improve_variable_names_task = tasks.improve_variable_names(variable_name_agent, game_request)
# add_code_comments_task = tasks.add_code_comments(variable_name_agent, improve_variable_names_task)
# add_documentation_task = tasks.add_documentation(variable_name_agent, improve_variable_names_task)

# CREATE CREW & ASSIGN TAS
crew = Crew(
    agents = [
        variable_name_agent,
#         documentation_agent
    ],
    tasks = [
        improve_variable_names_task,
#         add_documentation_task
    ],
    verbose=True
)

output = crew.kickoff()
end_time = time.perf_counter()

print("################################")
print("## Refactored Function Output ##")
print("################################")
print(f"## Time taken: {end_time - start_time:.2f} seconds\n")
print(output)
